# Faced_time
Time the face detector from [this repo](https://github.com/iitzco/faced?source=post_page---------------------------) which was discovered from [this great article](https://towardsdatascience.com/faced-cpu-real-time-face-detection-using-deep-learning-1488681c1602)

Actually use [this repo](https://github.com/Rysias/faced.git) which fixed an installation error

In [ ]:
import cv2
import os
import sys
import pdb
import imutils

from faced import FaceDetector
from faced.utils import annotate_image

face_detector = FaceDetector()

In [2]:
images_list = os.listdir('..//data//images')

img_path = '..//data//images//'

In [ ]:
img = cv2.imread(img_path + images_list[i])
rgb_img = cv2.cvtColor(img.copy(), cv2.COLOR_BGR2RGB)

# Receives RGB numpy image (HxWxC) and
# returns (x_center, y_center, width, height, confidence) tuples. 
bboxes = face_detector.predict(rgb_img, thresh=.85)
bboxes

In [3]:
# Use this utils function to annotate the image.
def annotate_and_show(img, bboxes, title='image'):
    ann_img = annotate_image(img, bboxes)
    cv2.imshow(title,ann_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Test the predictor
We want to get the accuracy and precision of the model for our 4-class classification problem.

In [18]:
# feed in rgb images
def predict_orientation(image, thresh=0.85, debug=False):
    def get_confidence(rgb_img):
#         pdb.set_trace()
        bboxes = face_detector.predict(rgb_img, thresh=thresh)
        if(len(bboxes)>1):
            annotate_and_show(rgb_img, bboxes, 'get_confidence')
        if(len(bboxes) < 1):
            return -1       # returns -1 if it doesn't find a face
        best_box = bboxes[0][4]
        if(len(bboxes) > 1):   # if we detect more than one face
            print(bboxes)
            annotate_and_show(rgb_img, bboxes, 'get_confidence')
            best_box = 0       # keep the max probability box (best_box)
            for box in bboxes:
                if(box[4] > best_box):
                    best_box = box[4]
        return best_box    # return the confidence
    
    # test vertical orientation:
    max_confidence = get_confidence(image)
    if(debug):
        print(max_confidence)
        
    if(max_confidence == -1):
        return_class = -1
    else:
        return_class = 0
        
    for angle in [90, 180, 270]:    # rotating clockwise
        temp = get_confidence(imutils.rotate(image, angle))
        if(temp > max_confidence):      # rotate image and get confidence
            max_confidence = temp
            return_class = 360-angle
           
    return return_class

Test a few images to make sure the function works

In [19]:
t = 5
print(images_list[t])
img = cv2.imread(img_path + imag es_list[t])
rgb_img = cv2.cvtColor(img.copy(), cv2.COLOR_BGR2RGB)
predict_orientation(rgb_img)

face_000205.jpg


0

## Get the Confusion matrix

In [7]:
def test_correct(o, image):
    if(o == 0 and len(image) < 16):
        return True
    if(str(o) == image[12:14]):
        return True
    if(str(o) == image[12:15]):
        return True
    return False

In [8]:
correct_class = 0
incorrect_class = 0
no_class = 0
total = len(images_list)
print("Testing " + str(total) + " images")

Testing 100 images


In [12]:
for image in images_list:
    img = cv2.imread(img_path + image)
    rgb_img = cv2.cvtColor(img.copy(), cv2.COLOR_BGR2RGB)
#     pdb.set_trace()
    orientation = predict_orientation(rgb_img)
    
    if(orientation == -1):
        no_class += 1
    elif(test_correct(orientation, image)):
        correct_class += 1
    else:
        incorrect_class += 1

[(256, 250, 149, 145, 0.99454486), (459, 246, 55, 90, 0.8662616)]
[(257, 250, 150, 145, 0.9943738), (459, 246, 55, 90, 0.86768675)]
[(256, 250, 150, 145, 0.99443626), (458, 244, 54, 89, 0.8618542)]
[(257, 250, 150, 145, 0.9945204), (459, 244, 54, 89, 0.86929214)]


In [16]:
print("Correct Class: {:.3f}\nIncorrect Class: {:.3f}\nNo Class: {:.3f}".format(correct_class/total, incorrect_class/total, no_class/total))

Correct Class: 0.760
Incorrect Class: 0.200
No Class: 0.040


In [ ]:
cv2.destroyAllWindows()

# Results
The pre-trained model scored 0.76 accuracy. I think I can do better with my own classifier :)